## OCR using Tesseract and EasyOCR

In [4]:
import cv2
import pytesseract
from PIL import Image

# Load and preprocess
image = cv2.imread("../uploads/images/page_1.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 11, 17, 17)  # Noise reduction
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Save cleaned image (optional)
cv2.imwrite("cleaned.jpg", thresh)
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\fkhaldi\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'
poppler_path = r'C:\Users\fkhaldi\Documents\SUPMTI\PFA\colab\Release-24.08.0-0\poppler-24.08.0\Library\bin'

# OCR
text = pytesseract.image_to_string(thresh, lang='ara')
print(text)

الذكاء االضطناعئ: أو الذكاء الصناعي: أو الذكاغ

الصطنعية! هو سلوك وخصائص معينة تتسم بها البرامج

الحاسوبية فتحاكي القدرات الذهنية البشرية وأنماط عملها. ومن
أهم هذه الخصائص: القدرة على التعلم واالستنتاجء ورد

الفعل على حاالت لم تبرمج في اآللة إال أنه مصطلح جدلي؟ لعدم
بأنه: نكاة ثبديه اآلالت والبرامج بما يحاكي القدرات الذهنية
البشرية وأنماط عملها مثل: القدرة على التعلم واالستنتاجء ورد
الفعل على حاالت لم تبرمج في اآللة وبه تصنغ حواسيب وبرامج
قادرة على اتخاذ سلوك بشري.



In [3]:
# !python -m pip install easyocr

import easyocr
import torch

if torch.cuda.is_available():
    print(f"GPU available: {torch.cuda.device_count()} devices")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available.")
reader = easyocr.Reader(['ar'])  # Arabic
results = reader.readtext('../uploads/images/page_1.jpg', paragraph=True, detail=0)
print("\n".join(results))

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


GPU is not available.
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

c:\Users\fkhaldi\Documents\SUPMTI\PFA\projet_pfa\aidoc\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


الذكاء الاصطناعي[ أو الذكاء الصناعي 2[ أو الذكاء الصتنعي ]3[ هو سلوك وخصائص معينة تتسم بها البر امج الحاسوا بية فتحاكي القدر ات الذهنية البشرية وأنماط عملها . ومن أهم هذه الخصائص : القدرة على التعلمء والا استنتاج  ورد الفعل علي حالات لم تبر مج في الآلةء إلا أنه مصطلح جدلي; لعدم وجود تعريف محدد للذكاء  ويمكن أن يعرف الذكاء الاصطناعي بأنه : ذكاء تبديه الآلات والبرا امج بما يحاكي القدر ات الذهنية البشرية وأنماط عملهاء مثل : القدرة على التعلمء و الاستنتاج  ورد الفعل على حالات لم تبر مج في الآلة وبه تصنع حواسيب وبرامج قادرة على اتخاذ سلوك بشري
